In [1]:
import sys
sys.path.insert(0,"..")

In [2]:
from langtest import Harness

In [3]:
!pip install johnsnowlabs

Defaulting to user installation because normal site-packages is not writeable


In [4]:
# Load test CoNLL
!wget https://raw.githubusercontent.com/JohnSnowLabs/langtest/main/langtest/data/conll/sample.conll

# Load train CoNLL
!wget https://raw.githubusercontent.com/JohnSnowLabs/langtest/main/demo/data/conll03.conll

'wget' is not recognized as an internal or external command,
operable program or batch file.
'wget' is not recognized as an internal or external command,
operable program or batch file.


In [5]:
from johnsnowlabs import nlp
spark = nlp.start()

🚨 Outdated Medical Secrets in license file. Version=5.0.2 but should be Version=4.4.3
🚨 Outdated OCR Secrets in license file. Version=5.0.0 but should be Version=4.4.2
📋 Loading license number 0 from C:\Users\devel/.johnsnowlabs\licenses/license_number_0_for_Spark-Healthcare_Spark-OCR.json
👷 Setting up  John Snow Labs home in C:\Users\devel/.johnsnowlabs, this might take a few minutes.
🙆 JSL Home setup in C:\Users\devel/.johnsnowlabs
🤓 Looks like you are missing some jars, trying fetching them ...
🚨 Cannot install 💊Spark-Healthcare because provided license file secret is outdated or it is invalid.
How to Fix this: 
Option1: Run nlp.install(force_browser=True) to get a Browser Window pop-up where you can refresh your license data
Option2: Run nlp.install(json_license_path="path/to/fresh_credentials.json") after downloading a fresh license from https://my.johnsnowlabs.com/subscriptions   
Option3: Run nlp.install(refresh_install=True,force_browser=True) to refresh everything 
Option4: Se

Exception: 🚨 Cannot install 💊Spark-Healthcare because provided license file secret is outdated or it is invalid.
How to Fix this: 
Option1: Run [92mnlp.install(force_browser=True)[39m to get a Browser Window pop-up where you can refresh your license data
Option2: Run [92mnlp.install(json_license_path="path/to/fresh_credentials.json")[39m after downloading a fresh license from https://my.johnsnowlabs.com/subscriptions   
Option3: Run [92mnlp.install(refresh_install=True,force_browser=True)[39m to refresh everything 
Option4: Set [92mnlp.settings.enforce_versions=False[39m and run[92m nlp.install(refresh_install=True,force_browser=True)[39m to disable this protection mechanism and try to install anyways. Not Recommended, can yield unforeseen consequences 


In [ ]:

embeddings = nlp.WordEmbeddingsModel.pretrained('glove_100d') \
		.setInputCols(["document", 'token']) \
		.setOutputCol("embeddings")

nerTagger = nlp.NerDLApproach()\
    .setInputCols(["document", "token", "embeddings"])\
    .setLabelColumn("label")\
    .setOutputCol("ner")\
    .setMaxEpochs(2)\
    .setBatchSize(64)\
    .setRandomSeed(0)\
    .setVerbose(1)\
    .setValidationSplit(0)\
    .setEvaluationLogExtended(True) \
    .setEnableOutputLogs(True)\
    .setIncludeConfidence(True)\
    .setOutputLogsPath('ner_logs')

training_pipeline = nlp.Pipeline(stages=[
          embeddings,
          nerTagger
 ])


conll_data = nlp.CoNLL().readDataset(spark, 'conll03.conll')

ner_model = training_pipeline.fit(conll_data)

ner_model.stages[-1].write().overwrite().save('models/trained_ner_model')|

In [ ]:
documentAssembler = nlp.DocumentAssembler()\
		.setInputCol("text")\
		.setOutputCol("document")

tokenizer = nlp.Tokenizer()\
		.setInputCols(["document"])\
		.setOutputCol("token")

embeddings = nlp.WordEmbeddingsModel.pretrained('glove_100d') \
		.setInputCols(["document", 'token']) \
		.setOutputCol("embeddings")

ner = nlp.NerDLModel.load("models/trained_ner_model") \
		.setInputCols(["document", "token", "embeddings"]) \
		.setOutputCol("ner")

ner_pipeline = nlp.Pipeline().setStages([
				documentAssembler,
				tokenizer,
				embeddings,
				ner
    ])

ner_model = ner_pipeline.fit(spark.createDataFrame([[""]]).toDF("text"))

In [ ]:
harness = Harness(task="ner", model={"model": ner_model, "hub": "johnsnowlabs"}, data={"data_source" :"data/conll/sample.conll"})

In [ ]:
harness.configure({
    'tests': {
        'defaults': {'min_pass_rate': 0.65},

        'robustness': {
            'lowercase': {'min_pass_rate': 0.60},
            'uppercase':{'min_pass_rate': 0.60}
        }
    }
})

In [ ]:
harness.generate(seed=42)

In [ ]:
harness.run()

In [ ]:
harness.report()

In [ ]:
data_kwargs = {
      "data_source" : "conll03.conll",
       }

harness.augment(training_data=data_kwargs, save_data_path="augmented_train.conll", export_mode="add")

In [ ]:
embeddings = nlp.WordEmbeddingsModel.pretrained('glove_100d') \
		.setInputCols(["document", 'token']) \
		.setOutputCol("embeddings")

nerTagger = nlp.NerDLApproach()\
    .setInputCols(["document", "token", "embeddings"])\
    .setLabelColumn("label")\
    .setOutputCol("ner")\
    .setMaxEpochs(2)\
    .setBatchSize(64)\
    .setRandomSeed(0)\
    .setVerbose(1)\
    .setValidationSplit(0)\
    .setEvaluationLogExtended(True) \
    .setEnableOutputLogs(True)\
    .setIncludeConfidence(True)\
    .setOutputLogsPath('ner_logs')

training_pipeline = nlp.Pipeline(stages=[
          embeddings,
          nerTagger
 ])


conll_data = nlp.CoNLL().readDataset(spark, 'augmented_train.conll')

ner_model = training_pipeline.fit(conll_data)

ner_model.stages[-1].write().overwrite().save('models/augmented_ner_model')

In [ ]:
documentAssembler = nlp.DocumentAssembler()\
		.setInputCol("text")\
		.setOutputCol("document")

tokenizer = nlp.Tokenizer()\
		.setInputCols(["document"])\
		.setOutputCol("token")

embeddings = nlp.WordEmbeddingsModel.pretrained('glove_100d') \
		.setInputCols(["document", 'token']) \
		.setOutputCol("embeddings")

ner = nlp.NerDLModel.load("models/augmented_ner_model") \
		.setInputCols(["document", "token", "embeddings"]) \
		.setOutputCol("ner")

ner_pipeline = nlp.Pipeline().setStages([
				documentAssembler,
				tokenizer,
				embeddings,
				ner
    ])

ner_model_2 = ner_pipeline.fit(spark.createDataFrame([[""]]).toDF("text"))

In [ ]:
harness = Harness(task="ner", model={"model": ner_model_2, "hub": "johnsnowlabs"}, data={"data_source" :"sample.conll"})

In [ ]:
harness.configure({
    'tests': {
        'defaults': {'min_pass_rate': 0.65},

        'robustness': {
            'lowercase': {'min_pass_rate': 0.60},
            'uppercase':{'min_pass_rate': 0.60}
        }
    }
})

In [ ]:
harness.generate(seed=42)

In [ ]:
harness.run().report()

In [ ]:
models = [{"model": ner_model , "hub":"johnsnowlabs"} , {"model":ner_model_2, "hub": "johnsnowlabs"}]

In [ ]:

harness = Harness(task="ner", model=models, data={"data_source":'sample.conll'})

In [ ]:
harness.configure({
    'tests': {
        'defaults': {'min_pass_rate': 0.65},

        'robustness': {
            'lowercase': {'min_pass_rate': 0.60},
            'uppercase':{'min_pass_rate': 0.60}
        }
    }
})

In [ ]:
harness.generate(seed=42)

In [ ]:
harness.run().report()

In [3]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")

In [9]:
model.config.model_type

't5'

In [3]:
from transformers import AutoModelForMaskedLM
model = AutoModelForMaskedLM.from_pretrained("bert-base-uncased")

In [13]:
model.config.name_or_path

'bert-base-uncased'

In [14]:
type(model)

transformers.models.bert.modeling_bert.BertForMaskedLM

In [31]:
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")

In [ ]:

import os

os.environ["OPENAI_API_KEY"] = "<YOUR_API_KEY>"

In [ ]:
harness = Harness(
                  task="question-answering", 
                  model={"model": model,"hub":"huggingface"}, 
                  data={"data_source" :"BoolQ",
                        "split":"test-tiny"}
                  )

In [4]:
harness = Harness(
    task="question-answering",
    model={"model": model, "hub":"huggingface"},
    data={"data_source" :"OpenBookQA",
                        "split":"test"},
    config={
         "evaluation": {"metric":"string_distance","distance":"jaro","threshold":0.1},
         'tests': {'defaults': {'min_pass_rate': 0.65},
           'robustness': {'uppercase': {'min_pass_rate': 0.75},
                        }
          }
 }
)

path BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementw

HFValidationError: Repo id must use alphanumeric chars or '-', '_', '.', '--' and '..' are forbidden, '-' and '.' cannot start or end the name, max length is 96: 'BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output)

In [27]:
type(model)

transformers.models.t5.modeling_t5.T5ForConditionalGeneration

In [17]:
model.config.name_or_path

'google/flan-t5-base'

In [18]:
print(type(model))

<class 'transformers.models.t5.modeling_t5.T5ForConditionalGeneration'>


In [25]:
if str(type(model)).split(".")[0]=="transformers":
    print("Yes")

In [24]:
type(model).split(".")[0]

AttributeError: type object 'T5ForConditionalGeneration' has no attribute 'split'

In [26]:
str(type(model)).split(".")[0]

"<class 'transformers"

In [28]:
print(type(model))

<class 'transformers.models.t5.modeling_t5.T5ForConditionalGeneration'>


In [29]:
# Example class definition
class MyModelClass:
    # Class definition goes here
    pass

# Assuming `model` is the object you want to check
model = MyModelClass()  # Example instantiation

# Check if `model` is an instance of MyModelClass
if isinstance(model, MyModelClass):
    print("model is an instance of MyModelClass")

model is an instance of MyModelClass


In [30]:
model